In [68]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [69]:

pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'e00ae563a9615c2ac56ab83fe4574a77', 'Date': 'Sat, 24 Aug 2024 20:54:13 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [70]:
import json

data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Susan White',
  'subject': 'CS 112 - Introduction to Computer Programming',
  'stars': 4.9,
  'review': 'Amazing professor, explains complex concepts clearly.'},
 {'professor': 'Dr. Mark Rogers',
  'subject': 'CS 330 - Formal Methods and Models',
  'stars': 4.7,
  'review': 'Engaging lectures and thorough understanding of formal methods.'},
 {'professor': 'Prof. Linda Thompson',
  'subject': 'CS 262 - Introduction to Low-Level Programming',
  'stars': 4.6,
  'review': 'Fantastic teaching style, makes complex systems easy to understand.'},
 {'professor': 'Dr. Michael Johnson',
  'subject': 'CS 455 - Computer Communications and Networking',
  'stars': 4.8,
  'review': 'Expert in networking, makes difficult topics accessible.'},
 {'professor': 'Prof. Emily Davis',
  'subject': 'CS 211 - Object-Oriented Programming',
  'stars': 4.7,
  'review': 'Very supportive and great at explaining object-oriented principles.'},
 {'professor': 'Dr. Richard Lee',
  'subject': 'CS 262 

In [74]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "Stars": review["stars"]
        }
    })

In [72]:
processed_data[0]


{'values': [-0.046755243,
  -0.023130676,
  -0.056413606,
  0.02636842,
  -0.006091352,
  -0.01282065,
  -0.0054979944,
  0.039923053,
  -0.018809108,
  -0.032103073,
  0.024735829,
  0.004259831,
  -0.02636842,
  -0.011668232,
  0.010138535,
  0.014611014,
  -0.03959379,
  0.010399201,
  0.038002357,
  0.0405267,
  0.015722275,
  -0.012813791,
  0.015269538,
  0.07655348,
  -0.038770635,
  -0.03759078,
  0.021799907,
  0.02454376,
  0.004479339,
  0.009342818,
  0.08182168,
  -0.0040986296,
  -0.0032017326,
  -0.06722438,
  -0.03438047,
  0.033145737,
  -0.020866998,
  0.0167375,
  0.0014345204,
  0.024969056,
  0.019536229,
  0.041596804,
  -0.029496413,
  -0.007140876,
  0.054438032,
  0.014103401,
  -0.073151104,
  -0.019467633,
  0.02282885,
  0.04812717,
  -0.032075636,
  -0.011386988,
  0.035971906,
  0.046041843,
  -0.055672765,
  0.06760852,
  0.035450574,
  0.027301332,
  -0.004030033,
  -0.027658032,
  0.04409371,
  0.0064206147,
  0.011167479,
  0.015873186,
  -0.03328293,


In [66]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 30}

In [73]:
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}